In [ ]:
def create_github_actions_pipeline():
    """創建 GitHub Actions CI/CD Pipeline"""
    
    print("🚀 創建 GitHub Actions CI/CD Pipeline...")
    
    workflows_dir = f"{GIT_REPO_DIR}/.github/workflows"
    
    # 主要 CI/CD 工作流程
    cicd_workflow = {
        "name": "Triton Inference CI/CD",
        "on": {
            "push": {
                "branches": ["main", "develop"]
            },
            "pull_request": {
                "branches": ["main"]
            },
            "release": {
                "types": ["created"]
            }
        },
        "env": {
            "DOCKER_REGISTRY": DOCKER_REGISTRY,
            "MODEL_REGISTRY": MODEL_REGISTRY,
            "PROJECT_NAME": PROJECT_NAME
        },
        "jobs": {
            "test": {
                "name": "運行測試套件",
                "runs-on": "ubuntu-latest",
                "strategy": {
                    "matrix": {
                        "python-version": ["3.8", "3.9", "3.10"]
                    }
                },
                "steps": [
                    {
                        "name": "Checkout 程式碼",
                        "uses": "actions/checkout@v4"
                    },
                    {
                        "name": "設置 Python ${{ matrix.python-version }}",
                        "uses": "actions/setup-python@v4",
                        "with": {
                            "python-version": "${{ matrix.python-version }}"
                        }
                    },
                    {
                        "name": "安裝依賴",
                        "run": "\\n".join([
                            "python -m pip install --upgrade pip",
                            "pip install pytest pytest-cov tritonclient[all]",
                            "pip install -r requirements.txt"
                        ])
                    },
                    {
                        "name": "運行單元測試",
                        "run": "pytest tests/unit/ -v --cov=src --cov-report=xml"
                    },
                    {
                        "name": "上傳測試覆蓋率",
                        "uses": "codecov/codecov-action@v3",
                        "with": {
                            "file": "./coverage.xml"
                        }
                    }
                ]
            },
            "security-scan": {
                "name": "安全掃描",
                "runs-on": "ubuntu-latest",
                "steps": [
                    {
                        "name": "Checkout 程式碼",
                        "uses": "actions/checkout@v4"
                    },
                    {
                        "name": "運行 Bandit 安全掃描",
                        "run": "\\n".join([
                            "pip install bandit",
                            "bandit -r src/ -f json -o bandit-report.json"
                        ])
                    },
                    {
                        "name": "運行 Safety 依賴掃描",
                        "run": "\\n".join([
                            "pip install safety",
                            "safety check --json --output safety-report.json"
                        ])
                    }
                ]
            },
            "build-and-push": {
                "name": "建置和推送 Docker 映像",
                "runs-on": "ubuntu-latest",
                "needs": ["test", "security-scan"],
                "if": "github.ref == 'refs/heads/main' || github.event_name == 'release'",
                "steps": [
                    {
                        "name": "Checkout 程式碼",
                        "uses": "actions/checkout@v4"
                    },
                    {
                        "name": "設置 Docker Buildx",
                        "uses": "docker/setup-buildx-action@v3"
                    },
                    {
                        "name": "登入 Docker Registry",
                        "uses": "docker/login-action@v3",
                        "with": {
                            "registry": "${{ env.DOCKER_REGISTRY }}",
                            "username": "${{ secrets.DOCKER_USERNAME }}",
                            "password": "${{ secrets.DOCKER_PASSWORD }}"
                        }
                    },
                    {
                        "name": "提取元數據",
                        "id": "meta",
                        "uses": "docker/metadata-action@v5",
                        "with": {
                            "images": "${{ env.DOCKER_REGISTRY }}/${{ env.PROJECT_NAME }}",
                            "tags": "\\n".join([
                                "type=ref,event=branch",
                                "type=ref,event=pr", 
                                "type=semver,pattern={{version}}",
                                "type=semver,pattern={{major}}.{{minor}}",
                                "type=sha,prefix={{branch}}-"
                            ])
                        }
                    },
                    {
                        "name": "建置和推送 Docker 映像",
                        "uses": "docker/build-push-action@v5",
                        "with": {
                            "context": ".",
                            "file": "./docker/Dockerfile",
                            "push": True,
                            "tags": "${{ steps.meta.outputs.tags }}",
                            "labels": "${{ steps.meta.outputs.labels }}",
                            "cache-from": "type=gha",
                            "cache-to": "type=gha,mode=max"
                        }
                    }
                ]
            },
            "model-validation": {
                "name": "模型驗證",
                "runs-on": "ubuntu-latest",
                "needs": ["build-and-push"],
                "if": "github.ref == 'refs/heads/main'",
                "steps": [
                    {
                        "name": "Checkout 程式碼", 
                        "uses": "actions/checkout@v4"
                    },
                    {
                        "name": "設置 Python",
                        "uses": "actions/setup-python@v4",
                        "with": {
                            "python-version": "3.10"
                        }
                    },
                    {
                        "name": "安裝依賴",
                        "run": "pip install tritonclient[all] requests numpy pandas"
                    },
                    {
                        "name": "運行模型驗證測試",
                        "run": "python tests/model-validation/validate_model.py",
                        "env": {
                            "MODEL_URI": "${{ env.MODEL_REGISTRY }}/models/llama2-7b-chat/latest",
                            "TRITON_URL": "localhost:8000"
                        }
                    }
                ]
            },
            "deploy-staging": {
                "name": "部署到測試環境",
                "runs-on": "ubuntu-latest", 
                "needs": ["model-validation"],
                "if": "github.ref == 'refs/heads/main'",
                "environment": "staging",
                "steps": [
                    {
                        "name": "Checkout 程式碼",
                        "uses": "actions/checkout@v4"
                    },
                    {
                        "name": "設置 kubectl",
                        "uses": "azure/setup-kubectl@v3",
                        "with": {
                            "version": "latest"
                        }
                    },
                    {
                        "name": "配置 kubeconfig",
                        "run": "echo '${{ secrets.KUBECONFIG_STAGING }}' | base64 -d > ~/.kube/config"
                    },
                    {
                        "name": "部署到 Staging",
                        "run": "\\n".join([
                            "kubectl apply -k k8s/overlays/staging",
                            "kubectl rollout status deployment/triton-server -n triton-staging --timeout=300s"
                        ])
                    },
                    {
                        "name": "運行整合測試",
                        "run": "python tests/integration/test_staging_deployment.py"
                    }
                ]
            },
            "deploy-production": {
                "name": "部署到生產環境",
                "runs-on": "ubuntu-latest",
                "needs": ["deploy-staging"], 
                "if": "github.event_name == 'release'",
                "environment": "production",
                "steps": [
                    {
                        "name": "Checkout 程式碼",
                        "uses": "actions/checkout@v4"
                    },
                    {
                        "name": "設置 kubectl",
                        "uses": "azure/setup-kubectl@v3"
                    },
                    {
                        "name": "配置 kubeconfig",
                        "run": "echo '${{ secrets.KUBECONFIG_PRODUCTION }}' | base64 -d > ~/.kube/config"
                    },
                    {
                        "name": "部署到 Production",
                        "run": "\\n".join([
                            "kubectl apply -k k8s/overlays/production",
                            "kubectl rollout status deployment/triton-server -n triton-production --timeout=600s"
                        ])
                    },
                    {
                        "name": "運行冒煙測試",
                        "run": "python tests/integration/test_production_smoke.py"
                    },
                    {
                        "name": "發送部署通知",
                        "uses": "8398a7/action-slack@v3",
                        "with": {
                            "status": "success",
                            "text": "🚀 Triton Server 已成功部署到生產環境 v${{ github.event.release.tag_name }}"
                        },
                        "env": {
                            "SLACK_WEBHOOK_URL": "${{ secrets.SLACK_WEBHOOK_URL }}"
                        }
                    }
                ]
            }
        }
    }
    
    with open(f"{workflows_dir}/cicd.yml", "w") as f:
        yaml.dump(cicd_workflow, f, default_flow_style=False, sort_keys=False)
    
    print("✅ GitHub Actions CI/CD Pipeline 創建完成")
    print(f"   工作流程文件: {workflows_dir}/cicd.yml")
    
    return cicd_workflow

# 創建 GitHub Actions Pipeline
github_workflow = create_github_actions_pipeline()

### 2.1 GitHub Actions CI/CD Pipeline"

## 🎯 實驗 2：自動化測試與驗證流程"

In [ ]:
def create_model_versioning_system():
    """創建模型版本管理系統"""

    print("🏷️  創建模型版本管理系統...")

    models_dir = f"{GIT_REPO_DIR}/models"

    # 模型註冊配置
    model_registry_config = {
        "apiVersion": "v1",
        "kind": "ConfigMap",
        "metadata": {
            "name": "model-registry-config",
            "labels": {
                "component": "model-registry"
            }
        },
        "data": {
            "registry_url": MODEL_REGISTRY,
            "default_stage": "staging",
            "auto_promote": "false",
            "validation_required": "true"
        }
    }

    with open(f"{models_dir}/registry-config.yaml", "w") as f:
        yaml.dump(model_registry_config, f, default_flow_style=False)

    # 模型元數據範例
    model_metadata = {
        "model_name": "llama2-7b-chat",
        "model_version": "v1.0.0",
        "model_stage": "production",
        "created_at": datetime.now().isoformat(),
        "model_uri": f"{MODEL_REGISTRY}/models/llama2-7b-chat/v1.0.0",
        "model_size_mb": 13500,
        "framework": "pytorch",
        "runtime": "python",
        "requirements": {
            "python": ">=3.8",
            "torch": ">=2.0.0",
            "transformers": ">=4.30.0"
        },
        "performance_metrics": {
            "accuracy": 0.92,
            "latency_p95_ms": 150,
            "throughput_rps": 100
        },
        "validation_tests": [
            "test_model_accuracy",
            "test_model_latency",
            "test_model_memory_usage"
        ],
        "deployment_config": {
            "min_replicas": 2,
            "max_replicas": 10,
            "target_cpu_utilization": 70,
            "gpu_required": True
        }
    }

    with open(f"{models_dir}/model-metadata.json", "w") as f:
        json.dump(model_metadata, f, indent=2)

    # 模型版本比較腳本
    version_comparison_script = '''#!/bin/bash
# 模型版本比較腳本

set -e

MODEL_NAME=$1
OLD_VERSION=$2
NEW_VERSION=$3

if [ $# -ne 3 ]; then
    echo "Usage: $0 <model_name> <old_version> <new_version>"
    exit 1
fi

echo "🔍 比較模型版本: $MODEL_NAME"
echo "   舊版本: $OLD_VERSION"
echo "   新版本: $NEW_VERSION"

# 下載模型元數據
OLD_METADATA=$(curl -s "${MODEL_REGISTRY}/models/${MODEL_NAME}/${OLD_VERSION}/metadata")
NEW_METADATA=$(curl -s "${MODEL_REGISTRY}/models/${MODEL_NAME}/${NEW_VERSION}/metadata")

# 比較性能指標
OLD_ACCURACY=$(echo $OLD_METADATA | jq -r '.performance_metrics.accuracy')
NEW_ACCURACY=$(echo $NEW_METADATA | jq -r '.performance_metrics.accuracy')

OLD_LATENCY=$(echo $OLD_METADATA | jq -r '.performance_metrics.latency_p95_ms')
NEW_LATENCY=$(echo $NEW_METADATA | jq -r '.performance_metrics.latency_p95_ms')

echo "📊 性能比較:"
echo "   準確率: $OLD_ACCURACY → $NEW_ACCURACY"
echo "   延遲 (P95): ${OLD_LATENCY}ms → ${NEW_LATENCY}ms"

# 計算改善幅度
ACCURACY_DIFF=$(echo "$NEW_ACCURACY - $OLD_ACCURACY" | bc -l)
LATENCY_DIFF=$(echo "$NEW_LATENCY - $OLD_LATENCY" | bc -l)

echo "📈 改善幅度:"
echo "   準確率變化: ${ACCURACY_DIFF}"
echo "   延遲變化: ${LATENCY_DIFF}ms"

# 決定是否建議升級
if (( $(echo "$ACCURACY_DIFF >= 0.01" | bc -l) )) && (( $(echo "$LATENCY_DIFF <= 10" | bc -l) )); then
    echo "✅ 建議升級到新版本"
    exit 0
else
    echo "⚠️  新版本未達升級標準"
    exit 1
fi
'''

    with open(f"{models_dir}/compare-versions.sh", "w") as f:
        f.write(version_comparison_script)

    # 設置執行權限
    os.chmod(f"{models_dir}/compare-versions.sh", 0o755)

    print("✅ 模型版本管理系統創建完成")
    print(f"   配置文件: {models_dir}/registry-config.yaml")
    print(f"   元數據範例: {models_dir}/model-metadata.json")
    print(f"   比較腳本: {models_dir}/compare-versions.sh")

    return model_metadata

# 創建模型版本管理
model_metadata = create_model_versioning_system()

### 1.3 模型版本管理系統"

In [ ]:
def create_environment_overlays():
    """創建環境特定的 Kustomize 覆蓋配置"""

    print("🌍 創建環境覆蓋配置...")

    overlays_config = {}

    for env in ENVIRONMENTS:
        overlay_dir = f"{GIT_REPO_DIR}/k8s/overlays/{env}"

        # 環境特定配置
        env_configs = {
            "dev": {
                "replicas": 1,
                "resources": {"cpu": "500m", "memory": "2Gi"},
                "image_tag": "dev",
                "namespace": "triton-dev"
            },
            "staging": {
                "replicas": 2,
                "resources": {"cpu": "1", "memory": "4Gi"},
                "image_tag": "staging",
                "namespace": "triton-staging"
            },
            "production": {
                "replicas": 3,
                "resources": {"cpu": "2", "memory": "8Gi"},
                "image_tag": "stable",
                "namespace": "triton-production"
            }
        }

        config = env_configs[env]

        # kustomization.yaml for overlay
        kustomization = {
            "apiVersion": "kustomize.config.k8s.io/v1beta1",
            "kind": "Kustomization",
            "metadata": {
                "name": f"triton-{env}"
            },
            "namespace": config["namespace"],
            "resources": [
                "../../base"
            ],
            "patchesStrategicMerge": [
                "deployment-patch.yaml"
            ],
            "images": [
                {
                    "name": "triton-server",
                    "newTag": config["image_tag"]
                }
            ],
            "replicas": [
                {
                    "name": "triton-server",
                    "count": config["replicas"]
                }
            ]
        }

        with open(f"{overlay_dir}/kustomization.yaml", "w") as f:
            yaml.dump(kustomization, f, default_flow_style=False)

        # deployment-patch.yaml
        deployment_patch = {
            "apiVersion": "apps/v1",
            "kind": "Deployment",
            "metadata": {
                "name": "triton-server"
            },
            "spec": {
                "template": {
                    "spec": {
                        "containers": [
                            {
                                "name": "triton-server",
                                "resources": {
                                    "requests": {
                                        "cpu": config["resources"]["cpu"],
                                        "memory": config["resources"]["memory"]
                                    },
                                    "limits": {
                                        "cpu": config["resources"]["cpu"],
                                        "memory": config["resources"]["memory"]
                                    }
                                },
                                "env": [
                                    {
                                        "name": "ENVIRONMENT",
                                        "value": env
                                    },
                                    {
                                        "name": "LOG_LEVEL",
                                        "value": "DEBUG" if env == "dev" else "INFO"
                                    }
                                ]
                            }
                        ]
                    }
                }
            }
        }

        with open(f"{overlay_dir}/deployment-patch.yaml", "w") as f:
            yaml.dump(deployment_patch, f, default_flow_style=False)

        overlays_config[env] = config
        print(f"   ✅ {env} 環境配置創建完成")

    print("✅ 所有環境覆蓋配置創建完成")
    return overlays_config

# 創建環境配置
overlays_config = create_environment_overlays()

# Lab 2.5.2 - CI/CD 與 MLOps 整合

## 🎯 實驗目標

本實驗將教您如何：
1. 設計完整的 GitOps 工作流程
2. 實現模型自動化部署流程
3. 建立多環境管理策略 (Dev/Staging/Prod)
4. 構建自動化測試與驗證 Pipeline
5. 實施模型品質門控與回滾機制

## 📋 前置需求

- 完成 Lab 2.5.1 (Kubernetes 部署)
- Git 版本控制系統
- GitHub Actions 或 GitLab CI/CD
- Docker Registry 訪問權限
- ArgoCD 或 Flux CD (GitOps)

---

## 📚 理論背景

### MLOps 與傳統 DevOps 的差異

**傳統 DevOps vs MLOps**:

| 維度 | 傳統 DevOps | MLOps |
|------|-------------|-------|
| **部署對象** | 應用程式碼 | 模型 + 程式碼 + 數據 |
| **版本管理** | Git 版本 | 模型版本 + Git 版本 |
| **測試策略** | 單元/整合測試 | 數據驗證 + 模型性能測試 |
| **監控重點** | 系統指標 | 模型漂移 + 預測品質 |
| **回滾機制** | 程式碼回滾 | 模型版本回滾 + A/B 測試 |
| **合規要求** | 安全掃描 | 模型可解釋性 + 偏見檢測 |

### AI 模型生命週期管理挑戰

**1. 模型版本複雜性**
- 模型權重檔案通常非常大 (GB 級別)
- 模型性能依賴於訓練數據和超參數
- 需要追蹤模型血緣和可重現性

**2. 多環境一致性**
- 訓練環境 vs 推理環境差異
- 硬體依賴 (GPU 型號、CUDA 版本)
- 推理延遲和吞吐量要求

**3. 模型品質保證**
- 自動化模型驗證和測試
- 性能回歸檢測
- 數據漂移監控

### 企業級 MLOps 架構

```
┌─────────────────────────────────────────────────────────────────┐
│                      Development Environment                    │
│  ┌───────────────┐  ┌──────────────┐  ┌──────────────────────┐ │
│  │ Data Science  │  │ Model Training│  │   Model Registry     │ │
│  │   Workspace   │→ │   Pipeline    │→ │  (MLflow/Neptune)    │ │
│  └───────────────┘  └──────────────┘  └──────────────────────┘ │
└─────────────────────────┬───────────────────────────────────────┘
                          │
                          ▼
┌─────────────────────────────────────────────────────────────────┐
│                      CI/CD Pipeline                             │
│  ┌───────────────┐  ┌──────────────┐  ┌──────────────────────┐ │
│  │   Git Trigger │→ │ Model Testing│→ │  Container Build     │ │
│  │   (Push/PR)   │  │ & Validation │  │  & Registry Push     │ │
│  └───────────────┘  └──────────────┘  └──────────────────────┘ │
└─────────────────────────┬───────────────────────────────────────┘
                          │
                          ▼
┌─────────────────────────────────────────────────────────────────┐
│                    GitOps Deployment                            │
│  ┌───────────────┐  ┌──────────────┐  ┌──────────────────────┐ │
│  │ Staging Env   │→ │ Production   │→ │   Monitoring &       │ │
│  │ (Auto Deploy) │  │ (Manual Gate)│  │   Alerting           │ │
│  └───────────────┘  └──────────────┘  └──────────────────────┘ │
└─────────────────────────────────────────────────────────────────┘
```

## 🛠️ 環境準備

In [ ]:
import os
import yaml
import json
import shutil
import subprocess
import time
import hashlib
import base64
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Any, Tuple
from pathlib import Path
from dataclasses import dataclass, field
from enum import Enum

import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# GitOps 和 CI/CD 工具
from git import Repo, GitCommandError
import docker

# 設置可視化樣式
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print(f"Environment ready at {datetime.now()}")
print(f"Working directory: {os.getcwd()}")

In [ ]:
# 設置 CI/CD 和 MLOps 環境
CICD_DIR = "/tmp/triton-mlops"
GIT_REPO_DIR = f"{CICD_DIR}/triton-inference-repo"
PIPELINE_DIR = f"{CICD_DIR}/pipelines"
ARGOCD_DIR = f"{CICD_DIR}/argocd"
ENVIRONMENTS_DIR = f"{CICD_DIR}/environments"
TESTS_DIR = f"{CICD_DIR}/tests"

# 創建目錄結構
directories = [
    CICD_DIR,
    GIT_REPO_DIR,
    PIPELINE_DIR,
    ARGOCD_DIR,
    f"{ENVIRONMENTS_DIR}/dev",
    f"{ENVIRONMENTS_DIR}/staging", 
    f"{ENVIRONMENTS_DIR}/production",
    f"{TESTS_DIR}/unit",
    f"{TESTS_DIR}/integration",
    f"{TESTS_DIR}/model-validation",
    f"{PIPELINE_DIR}/github-actions",
    f"{PIPELINE_DIR}/gitlab-ci",
    f"{PIPELINE_DIR}/jenkins"
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)

print("📁 CI/CD 和 MLOps 目錄結構:")
for directory in directories:
    print(f"   {directory}")

# 全域配置
PROJECT_NAME = "triton-inference"
DOCKER_REGISTRY = "your-registry.com"
MODEL_REGISTRY = "mlflow.your-company.com"
ENVIRONMENTS = ["dev", "staging", "production"]

print(f"\n⚙️  MLOps 配置:")
print(f"   專案名稱: {PROJECT_NAME}")
print(f"   Docker Registry: {DOCKER_REGISTRY}")
print(f"   模型註冊庫: {MODEL_REGISTRY}")
print(f"   環境: {', '.join(ENVIRONMENTS)}")

## 🎯 實驗 1：GitOps 工作流程設計

### 1.1 Git 倉庫結構設計

In [ ]:
def create_gitops_repository_structure():
    """創建 GitOps 倉庫結構"""
    
    print("📦 創建 GitOps 倉庫結構...")
    
    # 倉庫結構定義
    repo_structure = {
        ".github/workflows": "GitHub Actions 工作流程",
        "docker": "Docker 配置和 Dockerfile", 
        "k8s/base": "Kubernetes 基礎配置",
        "k8s/overlays/dev": "開發環境覆蓋配置",
        "k8s/overlays/staging": "測試環境覆蓋配置",
        "k8s/overlays/production": "生產環境覆蓋配置",
        "models": "模型配置和元數據",
        "tests": "自動化測試",
        "scripts": "部署和運維腳本",
        "docs": "文檔"
    }
    
    # 創建目錄結構
    for directory, description in repo_structure.items():
        full_path = f"{GIT_REPO_DIR}/{directory}"
        os.makedirs(full_path, exist_ok=True)
        
        # 創建 README.md
        readme_content = f"# {directory}\n\n{description}\n"
        with open(f"{full_path}/README.md", "w") as f:
            f.write(readme_content)
    
    # 創建根目錄 README.md
    root_readme = f"""
# Triton Inference Server - GitOps Repository

## 📋 概述

此倉庫包含 Triton Inference Server 的完整 GitOps 部署配置。

## 🏗️ 倉庫結構

```
triton-inference-repo/
├── .github/workflows/     # CI/CD 工作流程
├── docker/               # Docker 配置
├── k8s/                  # Kubernetes 配置
│   ├── base/             # 基礎配置 (Kustomize)
│   └── overlays/         # 環境特定配置
│       ├── dev/          # 開發環境
│       ├── staging/      # 測試環境
│       └── production/   # 生產環境
├── models/               # 模型配置和元數據
├── tests/                # 自動化測試
├── scripts/              # 部署腳本
└── docs/                 # 文檔
```

## 🚀 快速開始

### 開發環境部署
```bash
kubectl apply -k k8s/overlays/dev
```

### 生產環境部署
```bash
kubectl apply -k k8s/overlays/production
```

## 📚 工作流程

1. **開發**: 開發者提交程式碼和模型配置
2. **CI**: 自動測試和容器建置
3. **CD**: GitOps 自動部署到各環境
4. **監控**: 持續監控和告警

## 📖 更多文檔

- [部署指南](docs/deployment.md)
- [CI/CD 設置](docs/cicd.md)
- [故障排除](docs/troubleshooting.md)
"""
    
    with open(f"{GIT_REPO_DIR}/README.md", "w") as f:
        f.write(root_readme)
    
    print("✅ GitOps 倉庫結構創建完成")
    print(f"   位置: {GIT_REPO_DIR}")
    
    return repo_structure

# 創建倉庫結構
repo_structure = create_gitops_repository_structure()

# 顯示創建的結構
print("\n📂 創建的目錄結構:")
for directory, description in repo_structure.items():
    print(f"   {directory:<25} - {description}")

### 1.2 Kustomize 配置設計

In [ ]:
def create_kustomize_base_config():
    """創建 Kustomize 基礎配置"""
    
    print("⚙️  創建 Kustomize 基礎配置...")
    
    base_dir = f"{GIT_REPO_DIR}/k8s/base"
    
    # kustomization.yaml
    kustomization = {
        "apiVersion": "kustomize.config.k8s.io/v1beta1",
        "kind": "Kustomization",
        "metadata": {
            "name": "triton-base"
        },
        "resources": [
            "namespace.yaml",
            "serviceaccount.yaml",
            "configmap.yaml",
            "secret.yaml",
            "deployment.yaml",
            "service.yaml",
            "hpa.yaml"
        ],
        "commonLabels": {
            "app": "triton-server",
            "managed-by": "kustomize"
        },
        "images": [
            {
                "name": "triton-server",
                "newName": f"{DOCKER_REGISTRY}/triton-server",
                "newTag": "latest"
            }
        ]
    }
    
    with open(f"{base_dir}/kustomization.yaml", "w") as f:
        yaml.dump(kustomization, f, default_flow_style=False)
    
    # 基礎 deployment.yaml (簡化版)
    deployment = {
        "apiVersion": "apps/v1",
        "kind": "Deployment",
        "metadata": {
            "name": "triton-server",
            "labels": {
                "app": "triton-server"
            }
        },
        "spec": {
            "replicas": 1,
            "selector": {
                "matchLabels": {
                    "app": "triton-server"
                }
            },
            "template": {
                "metadata": {
                    "labels": {
                        "app": "triton-server"
                    }
                },
                "spec": {
                    "containers": [
                        {
                            "name": "triton-server",
                            "image": "triton-server:latest",
                            "ports": [
                                {"containerPort": 8000, "name": "http"},
                                {"containerPort": 8001, "name": "grpc"},
                                {"containerPort": 8002, "name": "metrics"}
                            ],
                            "resources": {
                                "requests": {
                                    "cpu": "1",
                                    "memory": "4Gi"
                                },
                                "limits": {
                                    "cpu": "2", 
                                    "memory": "8Gi"
                                }
                            }
                        }
                    ]
                }
            }
        }
    }
    
    with open(f"{base_dir}/deployment.yaml", "w") as f:
        yaml.dump(deployment, f, default_flow_style=False)
    
    # 基礎 service.yaml
    service = {
        "apiVersion": "v1",
        "kind": "Service",
        "metadata": {
            "name": "triton-server",
            "labels": {
                "app": "triton-server"
            }
        },
        "spec": {
            "selector": {
                "app": "triton-server"
            },
            "ports": [
                {"name": "http", "port": 8000, "targetPort": 8000},
                {"name": "grpc", "port": 8001, "targetPort": 8001},
                {"name": "metrics", "port": 8002, "targetPort": 8002}
            ],
            "type": "ClusterIP"
        }
    }
    
    with open(f"{base_dir}/service.yaml", "w") as f:
        yaml.dump(service, f, default_flow_style=False)
    
    print("✅ Kustomize 基礎配置創建完成")
    print(f"   位置: {base_dir}")
    
    return base_dir

# 創建基礎配置
base_config_dir = create_kustomize_base_config()